In [1]:
import pandas as pd
from datetime import datetime
import re

In [137]:
df = pd.read_csv('../data/absorbance/raw.csv')

# extracting the year, so that we can remove the wrong data

In [138]:
regex = '^\w+.\w+,\d{2}/\d{2}/(\d{4})$'

In [139]:
df['year'] = df[df.columns.values[0]].apply(lambda x: re.findall(regex, x)[0])

# dropping data with wrong date

In [140]:
rows_to_drop = df[df['year'] == '1601']
df.drop(rows_to_drop.index, inplace=True)

In [141]:
df.reset_index(drop=True, inplace=True)

# getting shelf life and treatment info

In [144]:
regex = '^\w*(T\d)(B\d)(\d)'

In [145]:
df['TREATMENT'] = df['Filename,'].apply(lambda x: re.findall(regex, x)[0][0])
df['BLOCK'] = df['Filename,'].apply(lambda x: re.findall(regex, x)[0][1])
df['SHELF-LIFE'] = df['Filename,'].apply(lambda x: re.findall(regex, x)[0][2])

# removing unnecessary info

In [147]:
cols = list(df.columns.values[:2])
cols.append('year')

In [148]:
df.drop(cols, axis=1, inplace=True)

# saving new data

In [149]:
df.to_csv('../data/absorbance/treated.csv')

In [170]:
df[df['TREATMENT'] == 'T3'].groupby(['SHELF-LIFE'])['BLOCK'].value_counts()

SHELF-LIFE  BLOCK
0           B1       24
            B2       24
            B3       24
1           B1       24
            B3       24
            B2       22
2           B1       24
3           B1       24
            B3       24
Name: BLOCK, dtype: int64

# appending the quality data

In [124]:
qlt = pd.read_csv('../data/quality/raw.csv')

In [125]:
qlt.drop(columns=['LEVELS'], inplace=True)
qlt['TREATMENT'] = qlt['TREATMENT'].apply(lambda x: str(x))
qlt['SHELF-LIFE'] = qlt['TREATMENT'].apply(lambda x: str(x))

In [126]:
df.set_index(['TREATMENT', 'SHELF-LIFE'], inplace=True)
qlt.set_index(['TREATMENT', 'SHELF-LIFE'], inplace=True)

In [76]:
qlt[['SST', 'TREATMENT', 'SHELF-LIFE']].transpose()

,0,1,2,3,4,5,6,7,8,9,...,820,821,822,823,824,825,826,827,828,829
SST,13.8,15.3,16.5,NaN,NaN,NaN,14.9,15.7,NaN,NaN,...,6.1,7.8,15.2,14,20.1,18.7,16.8,18.4,15.8,14
TREATMENT,T5,T5,T5,T5,T5,T5,T5,T5,T5,T5,...,T4,T4,T4,T4,T4,T4,T4,T4,T4,T4
SHELF-LIFE,0,0,0,0,0,0,0,0,0,0,...,3,3,3,3,3,3,3,3,3,3
